In [1]:
import tensorflow as tf #tf version 2.1.1
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tinymlgen import port
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
import pickle
import random
import math

C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
def hand_model_classifier(data,epoch_count, neurons):
    """
    Gets a labled dataset with measurments classified into 4 classes, the number of epochs to train and number of neurons per layer.
    Returns a classifier model trained with the first 70% of measurements and tested with the next 15%.
    """
    inp, output = [], []
    for tup in data:
        try:
            inp.append((tup[0],tup[1], tup[2],tup[3], tup[4],tup[5]))
            output.append(tup[6])
        except:
            print("problem tuple")
            pass
    size = min(len(inp), len(output))
    in1 = np.array(inp[:size])
    out = np.array(output[:size])

    # split into train, validation, test
    TRAIN_SPLIT =  int(0.6 * size)
    TEST_SPLIT = int(0.2 * size + TRAIN_SPLIT)
    x_train, x_test, x_validate = np.split(in1, [TRAIN_SPLIT, TEST_SPLIT])
    y_train, y_test, y_validate = np.split(out, [TRAIN_SPLIT, TEST_SPLIT])

    # create a NN with 3 layers of neurons
    model = Sequential([
    Dense(neurons, activation='relu', input_shape=(6,)),
    Dense(neurons, activation='relu'),
    Dense(neurons/2, activation='relu'),
    Dense(4, activation=tf.nn.softmax)
    ])

    model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=Adam(),
    metrics=['accuracy']
    )
    model.fit(x_train, y_train, verbose=0, epochs=epoch_count, batch_size=16,
                        validation_data=(x_validate, y_validate))
    
    return model

In [3]:
def create_data(path = r"C:\Users\Nadav\Downloads\log14-6_angle_{}.pkl"):
    """
    Function that takes a path to a folder with pickle files of measurements in four positions.
    The files named by the angles of the measurement angle as can be seen in the dataset folder on github.
    Returns a shuffled list of lists with the 2 sesnor meassurements and the label.
    """
    labled_data = []
    min_length = 100000
    for label in [0, 90, 180, 270]:
        with open(path.format(label), "rb") as f: 
            data = pickle.load(f)
            if len(data) < min_length:
                min_length = len(data)
    for label in [0, 90, 180, 270]:
        i = 0
        with open(path.format(label), "rb") as f: 
            data = pickle.load(f)
            for tup in data:
                unlabled = list(tup[1:])
                unlabled.append(int(label/90))
                labled_data.append(unlabled)
                i += 1
                if (i == min_length):
                    break
    random.shuffle(labled_data)
    return labled_data

In [4]:
def best_fit(tup):
    """
    Gets a tuple created by the model prediction and returns the index of the best fitting class.
    """
    i = 0
    index = 0
    max_val = -math.inf
    while (i < 4):
        if (tup[i] > max_val):
            index = i
            max_val = tup[i]
        i += 1
    return index

In [5]:
def error(model, test_data):
    """
    Gets a model and the raw data and uses the last 15% of entries as a validation set to check the accuracy.
    Returns the percentage of correct predictions.
    """
    inp, output = [], []
    for tup in test_data:
        try:
            inp.append((tup[0], tup[1],tup[2],tup[3], tup[4],tup[5]))
            output.append(tup[6])
        except:
            print("problem tuple")
            pass
    size = min(len(inp), len(output))
    in1 = np.array(inp[:size])
    out = np.array(output[:size])

    # split into train, validation, test
    TRAIN_SPLIT =  int(0.7 * size)
    TEST_SPLIT = int(0.15 * size + TRAIN_SPLIT)
    x_train, x_test, x_validate = np.split(in1, [TRAIN_SPLIT, TEST_SPLIT])
    y_train, y_test, y_validate = np.split(out, [TRAIN_SPLIT, TEST_SPLIT])
    
    tries = 0.0
    num_correct = 0.0
    
    for i in range(len(x_validate)):
        pred = model.predict(np.array([(x_validate[i][0],x_validate[i][1],x_validate[i][2],x_validate[i][3],x_validate[i][4],x_validate[i][5])]))[0]
        res = best_fit(pred)
        tries += 1
        if (res == y_validate[i]):
            num_correct += 1
    percent_correct = num_correct/tries*100
    print("percent correct: {}".format(percent_correct))
    return percent_correct

In [10]:
def main():
    labled = create_data()
    model = hand_model_classifier(labled, 100, 16)
    c_code = port(model, pretty_print=True).replace("model_data", "hand_model")
    model_str = "model_{}_{}_{}_classifier4_acc_{}".format(0, 100, 16, int (error(model, labled)))
    open(r"C:\Users\Nadav\Documents\IoT project\models\{}.h".format(model_str), "w").write(c_code) # hard coded path should be manually changed

percent correct: 42.87812041116006


24969